In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/LumiNet_Files/lumiply-colab/ #LumiNet 클론이 있는 디렉토리

/content/drive/MyDrive/LumiNet_Files/LumiNet-choi


In [ ]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
# ==========================================
# [Cell 1] 라이브러리 임포트 및 모델 엔진 설정 (실행 1회)
# ==========================================
import sys
import os
import torch
import cv2
import numpy as np
from PIL import Image
from omegaconf import OmegaConf
from ldm.util import instantiate_from_config
from cldm.model import load_state_dict
from cldm.ddim_hacked import DDIMSampler
import einops

# 경로 설정 (Colab 경로에 맞게 수정 필요)
BASE_MODEL_PATH = "./ckpt/trained_crossattn.ckpt" # 학습된 cross_attention ckpt
CONFIG_PATH = "./models/cldm_v21_LumiNet.yaml" # 모델생성

# 전역 변수 (모델을 메모리에 계속 유지하기 위함)
global_model = None
global_sampler = None
current_adapter_color = None

def initialize_engine():
    """베이스 모델을 메모리에 로드합니다."""
    global global_model, global_sampler

    if global_model is not None:
        print("✅ 모델이 이미 로드되어 있습니다.")
        return

    print("🚀 베이스 모델 로딩 시작... (최초 1회)")
    config = OmegaConf.load(CONFIG_PATH)
    model = instantiate_from_config(config.model).cpu()
    model.add_new_layers()

    if os.path.exists(BASE_MODEL_PATH):
        model.load_state_dict(load_state_dict(BASE_MODEL_PATH, location='cpu'), strict=False)
        print(f"📦 Base Model 로드 완료: {BASE_MODEL_PATH}")
    else:
        raise FileNotFoundError(f"Base Model 없음: {BASE_MODEL_PATH}")


    new_decoder = True # new_decoder를 사용 시 더 원본유지를 잘함
    if new_decoder:
        ae_checkpoint = "./ckpt/new_decoder.ckpt"
        model.change_first_stage(ae_checkpoint)

    model.cuda()
    model.eval()

    global_model = model
    global_sampler = DDIMSampler(model)
    print("✨ 엔진 준비 완료!")

def switch_adapter(target_color): # 모델 로드 후 Adaptor만 교체
    """요청된 색상으로 어댑터 가중치만 교체합니다."""
    global global_model, current_adapter_color

    if current_adapter_color == target_color:
        print(f"ℹ️ 이미 {target_color} 어댑터가 적용되어 있습니다.")
        return

    weights_path = f"./adaptors/adaptor_{target_color}.pth"

    if not os.path.exists(weights_path):
        print(f"(White 모드로 진행)") # Adaptor 교체를 하지않는경우 기본색상으로 진행
        current_adapter_color = "none"
        return

    print(f"🔄 어댑터 교체 중... ({current_adapter_color} -> {target_color})")
    checkpoint = torch.load(weights_path, map_location='cpu')

    # Hot-Swapping (모델을 끄지 않고 가중치만 덮어쓰기)
    if 'light_encoder' in checkpoint:
        global_model.control_model.prior_extracter.model_latents.light_encoder.load_state_dict(checkpoint['light_encoder'])
    if 'light_decoder' in checkpoint:
        global_model.control_model.prior_extracter.light_decoder.load_state_dict(checkpoint['light_decoder'])

    current_adapter_color = target_color
    print(f"✅ {target_color} 어댑터 장착 완료!")

def run_process(inference_root, color, guidance_scale=9.0, ddim_steps=50):
    """실제 추론을 수행하는 함수"""
    # 1. 어댑터 교체 시도
    switch_adapter(color)
    new_decoder = True # new_decoder를 사용 시 더 원본유지를 잘함

    print(f"\n📂 처리 시작: {inference_root} (Color: {color})")

    for root, dirs, files in os.walk(inference_root):
        off_name = next((f for f in files if 'off' in f.lower()), None)

        if off_name:
            off_path = os.path.join(root, off_name)
            save_path = os.path.join(root, f"output_{color}.jpg")
            print(f"▶ Processing: {os.path.basename(root)}")

            # 이미지 로드 & 전처리
            img_off = Image.open(off_path).convert("RGB") #off파일 불러오기
            orig_w, orig_h = img_off.size #원본 사이즈 저장
            img_off_resized = img_off.resize((512, 512), Image.BICUBIC) #off 이미지 리사이즈
            img_ref_resized = Image.new("RGB", (512, 512), (255, 255, 255)) # white reference 생성

            t_off = torch.from_numpy(np.array(img_off_resized).astype(np.float32)/255.0).permute(2,0,1).unsqueeze(0).cuda()
            t_ref = torch.from_numpy(np.array(img_ref_resized).astype(np.float32)/255.0).permute(2,0,1).unsqueeze(0).cuda()
            hint = torch.cat((t_off, t_ref), dim=1) #off, white 함께 묶여 hint의 형태로 control net으로 이동


            input_image_full = cv2.imread(off_path)[..., ::-1] / 255.0 #new_decoder용 off이미지 로드
            inp = cv2.resize(input_image_full, (512, 512))
            input_img = torch.from_numpy(inp.copy()).float().cuda()
            input_img = input_img.unsqueeze(0)
            input_img = einops.rearrange(input_img, 'b h w c -> b c h w').clone()

            # 추론
            with torch.no_grad():
                c_cat = hint # 원본(off) + white reference
                c = global_model.get_unconditional_conditioning(c_cat.shape[0]) #더미 텍스트 프롬프트
                cond = {"c_concat": [c_cat], "c_crossattn": [c]} # c_concat(이미지 힌트)은 ControlNet으로, c_crossattn(텍스트)은 Diffusion 모델로 전달
                shape = (4, 512 // 8, 512 // 8)

                samples, _ = global_sampler.sample(ddim_steps, 1, shape, cond, verbose=False,
                                            unconditional_guidance_scale=guidance_scale) #Diffusion에서 추론


                if new_decoder:
                    ae_hs = global_model.encode_first_stage(input_img * 2 - 1)[1]
                    x_sample = global_model.decode_new_first_stage(samples, ae_hs) # 원본 이미지의 특징(ae_hs)을 디코더에 함께 주입하여 디테일과 원본 구조를 복원함
                else:
                    x_sample = global_model.decode_first_stage(samples)
                x_sample = torch.clamp((x_sample + 1.0) / 2.0, min=0.0, max=1.0)

                result_numpy = x_sample.cpu().permute(0, 2, 3, 1).numpy()[0] * 255
                result_numpy = result_numpy.astype(np.uint8)
                result_final = cv2.resize(result_numpy, (orig_w, orig_h), interpolation=cv2.INTER_LANCZOS4) #원래off 크기로 리사이즈

                cv2.imwrite(save_path, cv2.cvtColor(result_final, cv2.COLOR_RGB2BGR))
                print(f"  💾 Saved: {save_path}")

# 초기화 실행
initialize_engine()

🚀 베이스 모델 로딩 시작... (최초 1회)


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/lightning.py:2058: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_tensor import pre_load_state_dict_hook, state_dict_hook


ControlLDM: Running in v-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 20 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, c

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 20 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is 1024 and using 20 heads.
Se

In [ ]:
# ==========================================
# [Cell 2] 추론 실행 (반복 실행 가능)
# ==========================================

# 1. 원하는 색상과 설정 입력
TARGET_COLOR = 'white'  # red, orange, yellow, green, blue, purple 원하는 색상 변경
INFERENCE_PATH = "./images/imgpair" # off가 있는 디렉토리

# 2. 실행 함수 호출
run_process(
    inference_root=INFERENCE_PATH,
    color=TARGET_COLOR,
    guidance_scale=9.0 # 입력 조건(힌트) 반영 강도
)

(White 모드로 진행)

📂 처리 시작: ./images/imgpair (Color: white)
▶ Processing: 95
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:06<00:00,  7.91it/s]


  💾 Saved: ./images/imgpair/lamp/95/output_white.jpg
▶ Processing: 61
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.90it/s]


  💾 Saved: ./images/imgpair/lamp/61/output_white.jpg
▶ Processing: 59
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


...

  💾 Saved: ./images/imgpair/pendant/7/output_white.jpg
▶ Processing: 8
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.99it/s]


  💾 Saved: ./images/imgpair/pendant/8/output_white.jpg
▶ Processing: 9
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.90it/s]


  💾 Saved: ./images/imgpair/pendant/9/output_white.jpg
